In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tqdm import tqdm
from IPython.display import HTML
# import line_profiler as lp 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable, functional
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import sys
sys.path.append('../src')
from model import *
from model2D import *
from datagen import *
from analysis import *
from datagen2D import *

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on GPU: ", device)
else:
    device = torch.device("cpu")
    print("Running on CPU")

torch.set_default_device(device)
# torch.cuda.synchronize()
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision("high")

Running on GPU:  cuda:0
Running on GPU:  cuda:0


/home/users/mattisdo/.pyenv/versions/3.10.12/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on GPU:  cuda:0
Running on GPU:  cuda:0


/home/users/mattisdo/.pyenv/versions/3.10.12/envs/env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/users/mattisdo/.pyenv/versions/3.10.12/envs/env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/users/mattisdo/.pyenv/vers

Running on GPU:  cuda:0


In [2]:
hidden_size = 4096
input_size = 1
output_size = 1

train1 = False

# Run multiple times to see the effect of the random initialization
if train1:
    model = torch_RNN_full_manual(input_size,200,output_size,hidden_size,lr=0.01,irnn=True,outputnn=True,bias=True,activation=True)
    _ = model.train_gradual(epochs=1000)
    model.plot_losses(average=30)
    # model.plot_accs()
    print(f"Init weights: Wh = {model.Wh_init}, Wx = {model.Wx_init}")
    # print("Guess on the first 15 training data: ",model(data[0:15]))
    print("Weight of the hidden layer Wh: ",model.rnn.weight_hh_l0, " Weight of the input layer Wx: ",model.rnn.weight_ih_l0)

In [6]:
class Datamodel(Dataset):
    def __init__(self,time_pos_points,labels):
        self.x = time_pos_points
        self.y = labels
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

n_data = 1000
t_steps = 10

data,labels = datagen_full_sum_normal(n_data,t_steps,normalize=True)
# data,labels = datagen_timewise_labels(n_data,t_steps,2)
# print(data[1],labels[1])

BATCH_SIZE = 16

training_data = Datamodel(data,labels)
training_loader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)

In [23]:
hidden_size = 100
input_size = 1
output_size = 1

train2 = True

if train2:
    # model2 = torch_RNN1(input_size,t_steps,output_size,hidden_size,lr=0.01,irnn=True,outputnn=False,Wx_normalize=True)
    model2 = torch_RNN_full_manual(input_size,t_steps,output_size,hidden_size,lr=0.001,irnn=False,outputnn=False,Wx_normalize=False,activation=False)
    print("Guess on the first 15 training data: ", model2(data[0:15]))
    _ = model2.train_loader(training_loader,epochs=300)
    model2.plot_losses(average=30)
    # model2.plot_accs()
    print("Guess on the first 15 training data: ", model2(data[0:15]))

Guess on the first 15 training data:  tensor([0.4809, 0.7412, 0.5998, 0.7195, 0.9161, 0.5020, 0.4521, 0.5075, 0.6997,
        0.5000, 0.6807, 0.9193, 0.5330, 0.8584, 0.6923], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)


  0%|          | 0/300 [00:00<?, ?it/s]


RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'